## 第一步，解释功能代码

In [2]:
import openai, load_api_key

In [3]:
# 对 stop 做了特殊的设置，只要连续两个换行或者类似连续两个换行的情况出现，就中止数据的生成。
# 这是避免模型一口气连测试代码也生成出来。那样的话，我们没法对测试代码的生成提出具体的要求。
def gpt35(prompt, model="text-davinci-002", temperature=0.4, max_tokens=1000,
          top_p=1, stop=["\n\n", "\n\t\n", "\n    \n"]):
    response = openai.Completion.create(
        model=model,
        prompt=prompt,
        temperature=temperature,
        max_tokens=max_tokens,
        top_p=top_p,
        stop=stop
    )
    message = response["choices"][0]["text"]
    return message


# 代码也可以让AI 生成
code = """
def format_time(seconds):
    minutes, seconds = divmod(seconds, 60)
    hours, minutes = divmod(minutes, 60)

    if hours > 0:
        return f"{hours}h{minutes}min{seconds}s"
    elif minutes > 0:
        return f"{minutes}min{seconds}s"
    else:
        return f"{seconds}s"
"""


def explain_code(function_to_test, unit_test_package="pytest"):
    prompt = f""""# How to write great unit tests with {unit_test_package}

In this advanced tutorial for experts, we'll use Python 3.10 and `{unit_test_package}` to write a suite of unit tests to verify the behavior of the following function.
```python
{function_to_test}
```

Before writing any unit tests, let's review what each element of the function is doing exactly and what the author's intentions may have been.
- First,"""
    response = gpt35(prompt)
    return response, prompt


code_explaination, prompt_to_explain_code = explain_code(code)
print(code_explaination)

 we use the `divmod` function to get the quotient and remainder of `seconds` divided by `60`. This gives us the number of minutes and seconds.
- Next, we do the same thing with minutes and hours.
- Finally, we use string formatting to return a string with the appropriate units.


## 第二步，设计测试用例

In [4]:
def generate_a_test_plan(full_code_explaination, unit_test_package="pytest"):
    prompt_to_explain_a_plan = f"""

A good unit test suite should aim to:
- Test the function's behavior for a wide range of possible inputs
- Test edge cases that the author may not have foreseen
- Take advantage of the features of `{unit_test_package}` to make the tests easy to write and maintain
- Be easy to read and understand, with clean code and descriptive names
- Be deterministic, so that the tests always pass or fail in the same way

`{unit_test_package}` has many convenient features that make it easy to write and maintain unit tests. We'll use them to write unit tests for the function above.

For this particular function, we'll want our unit tests to handle the following diverse scenarios (and under each scenario, we include a few examples as sub-bullets):
-"""
    prompt = full_code_explaination + prompt_to_explain_a_plan
    response = gpt35(prompt)
    return response, prompt


test_plan, prompt_to_get_test_plan = generate_a_test_plan(prompt_to_explain_code + code_explaination)
print(test_plan)

 Normal behavior:
    - `format_time(0)` should return `"0s"`
    - `format_time(1)` should return `"1s"`
    - `format_time(59)` should return `"59s"`
    - `format_time(60)` should return `"1min0s"`
    - `format_time(61)` should return `"1min1s"`
    - `format_time(3599)` should return `"59min59s"`
    - `format_time(3600)` should return `"1h0min0s"`
    - `format_time(3601)` should return `"1h0min1s"`
- Invalid inputs:
    - `format_time(None)` should raise a `TypeError`
    - `format_time("foo")` should raise a `TypeError`
    - `format_time(-1)` should raise a `ValueError`


## 第三步，撰写单元测试

In [5]:

def generate_test_cases(function_to_test, unit_test_package="pytest"):
    starter_comment = "Below, each test case is represented by a tuple passed to the @pytest.mark.parametrize decorator"
    prompt_to_generate_the_unit_test = f"""

Before going into the individual tests, let's first look at the complete suite of unit tests as a cohesive whole. We've added helpful comments to explain what each line does.
```python
import {unit_test_package}  # used for our unit tests

{function_to_test}

#{starter_comment}"""
    full_unit_test_prompt = prompt_to_explain_code + code_explaination + test_plan + prompt_to_generate_the_unit_test
    return gpt35(model="text-davinci-003", prompt=full_unit_test_prompt, stop="```"), prompt_to_generate_the_unit_test


unit_test_response, prompt_to_generate_the_unit_test = generate_test_cases(code)
print(unit_test_response)


@pytest.mark.parametrize("seconds,expected_result", [
    # Normal behavior
    (0, "0s"),
    (1, "1s"),
    (59, "59s"),
    (60, "1min0s"),
    (61, "1min1s"),
    (3599, "59min59s"),
    (3600, "1h0min0s"),
    (3601, "1h0min1s"),
    # Invalid inputs
    (None, TypeError),
    ("foo", TypeError),
    (-1, ValueError)
])
def test_format_time(seconds, expected_result):
    """
    This test verifies that the format_time function behaves as expected.
    """
    if isinstance(expected_result, type) and issubclass(expected_result, Exception):
        # If expected_result is an exception, we expect the function to raise it
        with pytest.raises(expected_result):
            format_time(seconds)
    else:
        # Otherwise, we expect the function to return the expected result
        assert format_time(seconds) == expected_result



In [ ]:
code_start_index = prompt_to_generate_the_unit_test.find("```python\n") + len("```python\n")
code_output = prompt_to_generate_the_unit_test[code_start_index:] + unit_test_response
# 打印完整代码
print(code_output)